<a href="https://colab.research.google.com/github/kittisaktaoma/colab_practice/blob/master/CalcSynergy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Library 



In [ ]:
!wget -c https://repo.continuum.io/miniconda/Miniconda3-py37_4.8.3-Linux-x86_64.sh
!chmod +x Miniconda3-py37_4.8.3-Linux-x86_64.sh
!time bash ./Miniconda3-py37_4.8.3-Linux-x86_64.sh -b -f -p /usr/local
!time conda install -q -y -c conda-forge rdkit
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

In [ ]:
! pip install keras
! pip install tensorflow
! pip install keras-gcn
! pip install numpy
! pip install matplotlib
! pip install scipy
! pip install deepchem
! pip install keras-tuner
! pip install chembl_webresource_client

In [17]:
#import deepchem as dc
import tensorflow as tf
import tensorflow.keras.layers as layers
import pandas as pd
import numpy as np
#import deepchem as dc
#from rdkit import Chem
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import BayesianOptimization
from chembl_webresource_client.new_client import new_client

#Boolean GRN

In [10]:
import numpy as np
def sim (r,c):
  return (np.random.randint(0, 2, size=(r,c)))

In [24]:
result = [sim(3,10),sim(5,10),sim(7,10),sim(3,10),sim(4,10),
          sim(6,10),sim(7,10),sim(3,10),sim(5,10),sim(1,10)]


[array([[0, 1, 0, 0, 1, 1, 1, 0, 1, 1],
        [0, 0, 1, 1, 1, 0, 1, 0, 1, 0],
        [1, 0, 0, 0, 0, 1, 0, 0, 1, 1]]),
 array([[1, 0, 1, 1, 1, 0, 1, 1, 0, 1],
        [0, 0, 1, 0, 0, 1, 1, 0, 0, 0],
        [0, 1, 0, 0, 1, 1, 1, 0, 0, 1],
        [1, 0, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 0, 0, 0, 1, 0, 1, 1, 0, 1]]),
 array([[1, 0, 0, 0, 1, 1, 0, 0, 0, 0],
        [1, 0, 1, 1, 0, 1, 1, 0, 0, 1],
        [0, 1, 0, 0, 0, 0, 1, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 0, 0, 1],
        [0, 1, 0, 1, 1, 1, 0, 1, 1, 1],
        [0, 0, 1, 0, 0, 1, 1, 0, 0, 1],
        [0, 1, 0, 1, 1, 0, 1, 0, 1, 1]]),
 array([[0, 0, 1, 0, 0, 0, 0, 1, 1, 0],
        [1, 1, 1, 0, 1, 0, 0, 1, 0, 1],
        [0, 1, 0, 1, 0, 0, 1, 0, 0, 0]]),
 array([[1, 0, 1, 0, 1, 1, 1, 0, 1, 0],
        [0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 1, 1, 1, 0, 0, 1, 0, 0],
        [1, 0, 1, 0, 1, 1, 0, 0, 0, 1]]),
 array([[0, 1, 1, 0, 0, 0, 0, 1, 1, 0],
        [1, 1, 1, 0, 0, 0, 1, 1, 0, 0],
        [0, 1, 1, 1, 0, 0, 1, 

In [29]:
from keras.preprocessing.sequence import pad_sequences
padded = pad_sequences(result,padding='post')
padded

array([[[0, 1, 0, 0, 1, 1, 1, 0, 1, 1],
        [0, 0, 1, 1, 1, 0, 1, 0, 1, 0],
        [1, 0, 0, 0, 0, 1, 0, 0, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],

       [[1, 0, 1, 1, 1, 0, 1, 1, 0, 1],
        [0, 0, 1, 0, 0, 1, 1, 0, 0, 0],
        [0, 1, 0, 0, 1, 1, 1, 0, 0, 1],
        [1, 0, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 0, 0, 0, 1, 0, 1, 1, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],

       [[1, 0, 0, 0, 1, 1, 0, 0, 0, 0],
        [1, 0, 1, 1, 0, 1, 1, 0, 0, 1],
        [0, 1, 0, 0, 0, 0, 1, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 0, 0, 1],
        [0, 1, 0, 1, 1, 1, 0, 1, 1, 1],
        [0, 0, 1, 0, 0, 1, 1, 0, 0, 1],
        [0, 1, 0, 1, 1, 0, 1, 0, 1, 1]],

       [[0, 0, 1, 0, 0, 0, 0, 1, 1, 0],
        [1, 1, 1, 0, 1, 0, 0, 1, 0, 1],
        [0, 1, 0, 1, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
model = Sequential()


# width and height are None because we want to process images of variable size 
# nb_channels is either 1 (grayscale) or 3 (rgb)
model.add(Convolution2D(32, 3, 3, input_shape=(nb_channels, None, None), border_mode='same'))

#Target Protein

## STITCH

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/kittisaktaoma/colab_practice/master/sample.txt",sep = "\t")
df_list_drug = pd.concat([df["drug_row"], df["drug_col"]]).unique().tolist()

In [ ]:
df.head

<bound method NDFrame.head of        block_id                 drug_row  ... cell_line_name  cellosaurus_accession
0         16145                     5-FU  ...        EFM192B              CVCL_1813
1         16146                     5-FU  ...        EFM192B              CVCL_1813
2         16147                     5-FU  ...        EFM192B              CVCL_1813
3         16148                     5-FU  ...        EFM192B              CVCL_1813
4         16149                     5-FU  ...        EFM192B              CVCL_1813
...         ...                      ...  ...            ...                    ...
45610    443886               VISMODEGIB  ...     MDA-MB-468              CVCL_0419
45611    443888               CRIZOTINIB  ...     MDA-MB-468              CVCL_0419
45612    443894              CABAZITAXEL  ...     MDA-MB-468              CVCL_0419
45613    443897               VANDETANIB  ...     MDA-MB-468              CVCL_0419
45614    443945  TOPOTECAN HYDROCHLORIDE  ... 

In [ ]:
def target_protein_stith(x):
  print(x)
  url = "http://stitch.embl.de/api/tsv/interactors?identifier="+x+"&species=9606"
  res = pd.read_csv(url,delimiter=".")
  res['Drug_name'] = x
  res_02 = res.iloc[1:,]
  return (res_02)

def get_result_stitch(x):
  try:
    return (target_protein_stith(x))
  except:
    pass
 
res_target_protein_stitch = pd.concat(map(get_result_stitch,df_list_drug))
res_target_protein_stitch

5-FU
ABT-888
AZD1775
BEZ-235
BORTEZOMIB
CARBOPLATIN
CYCLOPHOSPHAMIDE
DASATINIB
DEXAMETHASONE
DINACICLIB
DOXORUBICIN
ERLOTINIB
ETOPOSIDE
GELDANAMYCIN
GEMCITABINE
L778123
LAPATINIB
METFORMIN
METHOTREXATE
MITOMYCINE
MK-2206
MK-4541
MK-4827
MK-5108
MK-8669
MRK-003
OXALIPLATIN
PACLITAXEL
PD325901
SN-38
SORAFENIB
SUNITINIB
TEMOZOLOMIDE
TOPOTECAN
VINBLASTINE
VINORELBINE
ZOLINZA
CHEMBL17639
BUSULFAN
AZACYTIDINE
FLUDARABINE BASE
TENIPOSIDE
NSC733504
EXEMESTANE
BLEOMYCIN
ANTIBIOTIC AD 32
MITOXANTRONE
5-FLUORO-2'-DEOXYURIDINE
MITOTANE
METHOXSALEN
FULVESTRANT
CARMUSTINE
ADM HYDROCHLORIDE
TAMOXIFEN CITRATE
CHEMBL277800
ALLOPURINOL
THIOTEPA
MERCAPTOPURINE
ANTIBIOTIC AY 22989
IFOSFAMIDE
ANASTROZOLE
VINCRISTINE SULFATE
VINBLASTINE SULFATE
IMATINIB
IMIQUIMOD
DACARBAZINE
ZOLEDRONIC ACID
QUINACRINE HYDROCHLORIDE
NILOTINIB
BENDAMUSTINE HYDROCHLORIDE
LETROZOLE
ALTRETAMINE
DOCETAXEL
DEXRAZOXANE
RALOXIFENE
CYTARABINE HYDROCHLORIDE
34793-34-5
MECHLORETHAMINE HYDROCHLORIDE
ACTINOMYCIN D
HYDROXYUREA
PROCARBAZIN

,itemId,Drug_name
9606,ENSP00000359211,5-FU
9606,ENSP00000315644,5-FU
9606,ENSP00000330032,5-FU
9606,ENSP00000252029,5-FU
9606,ENSP00000269305,5-FU
...,...,...
9606,ENSP00000358747,NSC256439
9606,ENSP00000356198,NSC256439
9606,ENSP00000341828,NSC256439
9606,ENSP00000388648,MK-8776


In [ ]:
len(res_target_protein_stitch.iloc[:,1].unique())

89

In [ ]:
tartget_protein_df = pd.crosstab(index=res_target_protein_stitch.iloc[:,1], 
            columns=res_target_protein_stitch.iloc[:,0])

tartget_protein_df

itemId,ENSP00000001146,ENSP00000013807,ENSP00000037502,ENSP00000080059,ENSP00000206249,ENSP00000215832,ENSP00000216714,ENSP00000216911,ENSP00000217133,ENSP00000217185,ENSP00000217426,ENSP00000218032,ENSP00000219431,ENSP00000219700,ENSP00000220764,ENSP00000220809,ENSP00000221130,ENSP00000221930,ENSP00000222390,ENSP00000222982,ENSP00000223095,ENSP00000225275,ENSP00000225577,ENSP00000226413,ENSP00000226730,ENSP00000227507,ENSP00000227758,ENSP00000228682,ENSP00000229239,ENSP00000231449,ENSP00000231509,ENSP00000231790,ENSP00000231948,ENSP00000233146,ENSP00000237612,ENSP00000239849,ENSP00000241052,ENSP00000241337,ENSP00000241453,ENSP00000242576,...,ENSP00000384408,ENSP00000384957,ENSP00000385721,ENSP00000386884,ENSP00000387230,ENSP00000387699,ENSP00000388648,ENSP00000388902,ENSP00000391676,ENSP00000393583,ENSP00000394932,ENSP00000396308,ENSP00000396704,ENSP00000398698,ENSP00000399985,ENSP00000402084,ENSP00000408632,ENSP00000408860,ENSP00000410076,ENSP00000411532,ENSP00000412237,ENSP00000414334,ENSP00000417281,ENSP00000417517,ENSP00000418532,ENSP00000418960,ENSP00000419692,ENSP00000419782,ENSP00000420176,ENSP00000427562,ENSP00000430432,ENSP00000434516,ENSP00000441927,ENSP00000447803,ENSP00000448165,ENSP00000457706,ENSP00000459962,ENSP00000462196,ENSP00000463999,ENSP00000464149
Drug_name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5-FLUORO-2'-DEOXYURIDINE,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
5-FU,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ABIRATERONE,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ABT-888,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ALLOPURINOL,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VEMURAFENIB,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
VINBLASTINE,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
VINORELBINE,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
len(remain_df_04.iloc[:,1].unique())

87

#Drug Structure (GCN layer)

In [ ]:
def tune_nn_model(hp):
    model = keras.Sequential()
    
    drop_rate = hp.Choice('drop_rate',
                            [
                              0.0,
                              0.1,
                              0.2,
                              0.3,
                              0.4,
                              0.5,
                              0.6,
                              0.7,
                              0.8,
                              0.9
                            ])

     
    model.add(keras.layers.Dense(
      units=28,
      activation="relu",
      input_shape=[df_x.shape[1]]
    ))
    model.add(keras.layers.Dropout(rate=drop_rate))
    model.add(keras.layers.Dense(16, activation='relu'))
    model.add(keras.layers.Dropout(rate=drop_rate))
    model.add(keras.layers.Dense(1, activation='sigmoid')) 
    lr = hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
    momentum = hp.Choice('momentum', [0.0, 0.2, 0.4, 0.6, 0.8, 0.9])

    model.compile(
        optimizer=keras.optimizers.SGD(lr, momentum=momentum),
        loss = 'binary_crossentropy',
        metrics = ['accuracy'])
    return model

In [ ]:
b_tuner = BayesianOptimization(
    tune_nn_model,
    objective='accuracy',
    max_trials=20,
    executions_per_trial=5,
    directory='test_dir_03',
    project_name='b_tune_nn'
)

INFO:tensorflow:Reloading Oracle from existing project test_dir_03/b_tune_nn/oracle.json


In [ ]:
df_x_x = df_x.to_numpy()
df_y_y = df_y.to_numpy()

In [ ]:
b_tuner.search(df_x_x,df_y_y)

In [ ]:
b_tuner.results_summary()

In [ ]:
# Retrieve the best model and display its architecture
best_model = b_tuner.get_best_models(num_models=1)[0]
best_model.summary()

### Remaining Drug

In [ ]:
#filterremaining drug from dataset
remaining_drug = res_target_protein_stitch['Drug_name'].unique().tolist()
remain_df_01 = df[df["drug_row"].isin(remaining_drug)]
remain_df_02 = remain_df_01[remain_df_01["drug_col"].isin(remaining_drug)]
remain_df_03 = remain_df_02[['cell_line_name','drug_row','drug_col','synergy_loewe']]
remain_df_04 = remain_df_03.groupby(['cell_line_name','drug_row','drug_col']).agg({'synergy_loewe': 'mean'}).reset_index()
remain_df_04


,cell_line_name,drug_row,drug_col,synergy_loewe
0,BT-549,5-FLUORO-2'-DEOXYURIDINE,ALLOPURINOL,-0.4400
1,BT-549,5-FLUORO-2'-DEOXYURIDINE,AMIFOSTINE,0.2300
2,BT-549,5-FLUORO-2'-DEOXYURIDINE,ANASTROZOLE,-2.5300
3,BT-549,5-FLUORO-2'-DEOXYURIDINE,AZACYTIDINE,2.1500
4,BT-549,5-FLUORO-2'-DEOXYURIDINE,BLEOMYCIN,-0.2000
...,...,...,...,...
16345,ZR751,ZOLINZA,SN-38,0.3000
16346,ZR751,ZOLINZA,SORAFENIB,1.8425
16347,ZR751,ZOLINZA,SUNITINIB,-9.8175
16348,ZR751,ZOLINZA,TEMOZOLOMIDE,-67.0150


#Concatete data

##target protein

In [ ]:
x_tmp = tartget_protein_df.unstack()
df_tmp = pd.DataFrame(x_tmp.0.str.split(' ',1).tolist(),
                                 columns = ['flips','row'])


0
itemId          Drug_name                  
ENSP00000001146 5-FLUORO-2'-DEOXYURIDINE  0
                5-FU                      0
                ABIRATERONE               0
                ABT-888                   0
                ALLOPURINOL               0
...                                      ..
ENSP00000464149 VEMURAFENIB               0
                VINBLASTINE               0
                VINORELBINE               0
                VISMODEGIB                0
                ZOLINZA                   0

[38804 rows x 1 columns]

In [ ]:
pd.merge(remain_df_04, climate_precip, on=["STATION", "DATE"])

,cell_line_name,drug_row,drug_col,synergy_loewe
0,BT-549,5-FLUORO-2'-DEOXYURIDINE,ALLOPURINOL,-0.4400
1,BT-549,5-FLUORO-2'-DEOXYURIDINE,AMIFOSTINE,0.2300
2,BT-549,5-FLUORO-2'-DEOXYURIDINE,ANASTROZOLE,-2.5300
3,BT-549,5-FLUORO-2'-DEOXYURIDINE,AZACYTIDINE,2.1500
4,BT-549,5-FLUORO-2'-DEOXYURIDINE,BLEOMYCIN,-0.2000
...,...,...,...,...
16345,ZR751,ZOLINZA,SN-38,0.3000
16346,ZR751,ZOLINZA,SORAFENIB,1.8425
16347,ZR751,ZOLINZA,SUNITINIB,-9.8175
16348,ZR751,ZOLINZA,TEMOZOLOMIDE,-67.0150
